## Import libraries

In [1]:
# !pip install tensorly

import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from math import ceil
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import torch.nn.init as init

In [2]:
"""
5 runs of 50 epochs, seed = 10, 20, 30, 40, 50;
validation accuracies: 0.9492, 0.9457, 0.9463, 0.9439, 0.9455
"""
#from __future__ import print_function, division
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import balanced_accuracy_score

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

dtype = torch.float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


## train data processing

In [3]:
df_train_1 = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_training.csv')
df_train_2 = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_validation.csv')
# concat train and validation into train. we don't need validation set
df_train = df_train_1.append(df_train_2).reset_index(drop=True)

# undersampling for "negative" samples
negative_df = df_train[df_train['label'] == 'Negative']
positive_df = df_train[df_train['label'] == 'Positive']

# Get number of "positive" samples
num_positive = len(positive_df)

# take the same number of "negative" samples as there are "positive" samples
balanced_negative_df = negative_df.sample(n=num_positive, random_state=10086)

df_train_balanced = pd.concat([positive_df, balanced_negative_df])
print(df_train_balanced['label'].value_counts())

Positive    4057
Negative    4057
Name: label, dtype: int64


In [4]:
pd_X_train = df_train_balanced.iloc[:, 5:]
pd_y_train = df_train_balanced.iloc[:, 0]

N = len(pd_X_train)
K = 2

pd_X_train = pd_X_train.values
X_train = torch.tensor(pd_X_train, dtype=dtype, device=device)
X_train = torch.t(X_train)

# scaler = MinMaxScaler()
# x = pd_X_train.values
# x_scaled = scaler.fit_transform(x)
# X_train = torch.tensor(x_scaled, dtype=dtype, device=device)
# X_train = torch.t(X_train)

# Initialize the LabelEncoder
encoder = LabelEncoder()
# Fit and transform the y values
y = encoder.fit_transform(pd_y_train.values)
y_train=torch.tensor(y, dtype=torch.long, device=device)
y_train = torch.flatten(y_train)

y_one_hot = torch.zeros(N, K, device=device).scatter_(1, y_train.unsqueeze(1), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)

print(list(encoder.classes_))

['Negative', 'Positive']


## test data processing

In [5]:
df_test = pd.read_csv('/home/c/cl237/Datasets/Flare_LSTM_dataset/M/normalized_testing.csv')

# Undersampling for "negative" samples
#negative_df = df_test[df_test['label'] == 'Negative']
#positive_df = df_test[df_test['label'] == 'Positive']

# Get number of "positive" samples
#num_positive = len(positive_df)

# Take the same number of "negative" samples as there are "positive" samples
#balanced_negative_df = negative_df.sample(n=num_positive, random_state=10086)

#df_test_balanced = pd.concat([positive_df, balanced_negative_df])
#print(df_test_balanced['label'].value_counts())

In [6]:
pd_X_test = df_test.iloc[:, 5:]
pd_y_test = df_test.iloc[:, 0]

N_test = len(pd_X_test)
K = 2

pd_X_test = pd_X_test.values
X_test = torch.tensor(pd_X_test, dtype=dtype, device=device)
X_test = torch.t(X_test)
# x = pd_X_test.values
# x_scaled = scaler.transform(x)  # only transform x, don't fit the scaler again
# X_test = torch.tensor(x_scaled, dtype=dtype, device=device)
# X_test = torch.t(X_test)


# Initialize the LabelEncoder
encoder = LabelEncoder()
# Fit and transform the y values
y = encoder.fit_transform(pd_y_test.values)
y_test=torch.tensor(y, dtype=torch.long, device=device)
y_test = torch.flatten(y_test)

y_test_one_hot = torch.zeros(N_test, K, device=device).scatter_(1, y_test.unsqueeze(1), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)

In [7]:
print(df_test['label'].value_counts())

Negative    43411
Positive     1278
Name: label, dtype: int64


## Main algorithm

### Define functions for updating blocks

In [8]:
# def updateMask(W, sparsity):
#     torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy()
#     Mask
#     Wsparse
#     return Mask, Wsparse

# Wsquare = torch.square(W3)
# Threshold = torch.quantile(torch.reshape(Wsquare, (-1,)), 0.5, interpolation='linear')
# Wsparse = W3
# Wsparse[Wsquare < Threshold] =  0

# Wsparse

In [9]:
def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device)
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    return Vstar

In [10]:
def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [11]:
def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [12]:
def updateWbsparse(U, V, W, b,  W_tensor2matrix, alpha, rho,tau):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

### Define the proximal operator of the ReLU activation function

In [13]:
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

### Effective Sparsity

In [14]:
def process_weights(W1, W2, W3):
    # Clone the tensors to keep the originals unchanged
    W1, W2, W3 = W1.clone(), W2.clone(), W3.clone()

    while True:
        # Store a copy of the current weights
        old_W1, old_W2, old_W3 = W1.clone(), W2.clone(), W3.clone()

        # Check all rows of W1
        # If all values in a row of W1 are 0, set corresponding column in W2 to 0
        zero_rows_W1 = torch.all(W1 == 0, dim=1)
        W2[:, zero_rows_W1] = 0

        # Check all columns of W2
        # If all values in a column of W2 are 0, set corresponding row in W1 to 0
        zero_cols_W2 = torch.all(W2 == 0, dim=0)
        W1[zero_cols_W2, :] = 0

        # Check all rows of W2
        # If all values in a row of W2 are 0, set corresponding column in W3 to 0
        zero_rows_W2 = torch.all(W2 == 0, dim=1)
        W3[:, zero_rows_W2] = 0

        # Check if matrices are unchanged
        if torch.equal(W1, old_W1) and torch.equal(W2, old_W2) and torch.equal(W3, old_W3):
            break

    return W1, W2, W3

### Parameter initialization

In [20]:
#df = pd.DataFrame()
#df.to_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv')

niter = 1000
sparsity = 0.9

tau = 200
alpha = 1
rho = 1
gamma = 100

        

# Do something with these parameters
print(sparsity, tau, gamma, rho, alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)
bacc_train = np.empty(niter)
bacc_test = np.empty(niter)
true_sparsity = np.empty(niter)
effective_sparsity = np.empty(niter)

results = torch.zeros(1, 9, niter)

for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 40
    d1 =  300
    d2 =  100
    d3 = K # Layers: input + 2 hidden + output

    W1 = 0.2*init.orthogonal_(torch.empty(d1, d0, device=device,dtype=dtype), gain=1.0)
    b1 = 0*torch.ones(d1, 1, dtype=torch.float64,device=device)
    W1square = torch.square(W1)
    Threshold = torch.quantile(torch.reshape(W1square, (-1,)), sparsity, interpolation='linear')
    W1sparse = W1
    W1sparse[W1square < Threshold] =  0

    W2 = 0.2*init.orthogonal_(torch.empty(d2, d1, device=device,dtype=dtype), gain=1.0)
    b2 = 0*torch.ones(d2, 1, dtype=torch.float64,device=device)
    W2square = torch.square(W2)
    Threshold = torch.quantile(torch.reshape(W2square, (-1,)), sparsity, interpolation='linear')
    W2sparse = W2
    W2sparse[W2square < Threshold] =  0


    W3 = 0.2*init.orthogonal_(torch.empty(d3, d2, device=device,dtype=dtype), gain=1.0)
    b3 = 0*torch.ones(d3, 1, dtype=torch.float64,device=device)
    # W3square = torch.square(W3)
    # Threshold = torch.quantile(torch.reshape(W3square, (-1,)), sparsity, interpolation='linear')
    # W3sparse = W3
    # W3sparse[W3square < Threshold] =  0


    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = U3
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4


    print('Train on', N, 'samples, validate on', N_test, 'samples')

    for k in range(niter):

        start = time.time()

        # update V4
        V3 = (y_one_hot + gamma*U3 + alpha*V3)/(1 + gamma + alpha)

        # update U4
        U3 = (gamma*V3 + rho*(torch.mm(W3,V2) + b3.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W3, b3 = updateWb_org(U3,V2,W3,b3, alpha,rho)
        # W3square = torch.square(W3)
        # Threshold = torch.quantile(torch.reshape(W3square, (-1,)), sparsity, interpolation='linear')
        # W3sparse = W3
        # W3sparse[W3square < Threshold] =  0
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2

        W2, b2 = updateWbsparse(U2,V1,W2,b2,W2sparse, alpha,rho,tau)
        W2square = torch.square(W2)
        Threshold = torch.quantile(torch.reshape(W2square, (-1,)), sparsity, interpolation='linear')
        W2sparse = W2
        W2sparse[W2square < Threshold] =  0

        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWbsparse(U1,X_train,W1,b1,W1sparse, alpha,rho,tau)
        W1square = torch.square(W1)
        Threshold = torch.quantile(torch.reshape(W1square, (-1,)), sparsity, interpolation='linear')
        W1sparse = W1
        W1sparse[W1square < Threshold] =  0

        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1sparse, X_train)).double()
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N),  W2sparse, a1_train)).double()
        pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1sparse, X_test)).double()
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test),  W2sparse, a1_test)).double()
        pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)


        loss1[k] = gamma/2*torch.pow(torch.dist(V3,y_one_hot,2),2).cpu().numpy()
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1sparse, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N),  W2sparse, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,U3,2),2).cpu().numpy() \
        + tau/2*torch.pow(torch.dist(W1,W1sparse,2),2).cpu().numpy()\
        + tau/2*torch.pow(torch.dist(W2,W2sparse,2),2).cpu().numpy()\
        # + tau/2*torch.pow(torch.dist(W3,W3sparse,2),2).cpu().numpy()

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())


        pred_train_np = pred.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        pred_test_np = pred_test.cpu().numpy()
        y_test_np = y_test.cpu().numpy()

        bacc_train[k] = balanced_accuracy_score(y_train_np, pred_train_np)
        bacc_test[k] = balanced_accuracy_score(y_test_np, pred_test_np)

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # sparsity
        num_zeros_W1 = torch.sum(W1 == 0).item()
        num_zeros_W2 = torch.sum(W2 == 0).item()
        num_zeros_W3 = torch.sum(W3 == 0).item()
        total_zeros_old = num_zeros_W1 + num_zeros_W2 + num_zeros_W3
        total_weights = d0*d1+d1*d2+d2*d3
        true_sparsity[k] = total_zeros_old / total_weights

        new_W1, new_W2, new_W3 = process_weights(W1, W2, W3)
        num_zeros_W1_new = torch.sum(new_W1 == 0).item()
        num_zeros_W2_new = torch.sum(new_W2 == 0).item()
        num_zeros_W3_new = torch.sum(new_W3 == 0).item()
        total_zeros = num_zeros_W1_new + num_zeros_W2_new + num_zeros_W3_new
        total_weights = d0*d1+d1*d2+d2*d3
        effective_sparsity[k] = total_zeros / total_weights


        # print results
        print('Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k],
              '-', 'bacc_train:', bacc_train[k], '-', 'bacc_test:', bacc_test[k],'-', 'true_sparsity:', true_sparsity[k],
             '-', 'effective_sparsity:', effective_sparsity[k])


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    results[Out_iter,5,:] = torch.tensor(bacc_train)
    results[Out_iter,6,:] = torch.tensor(bacc_test)
    results[Out_iter,7,:] = torch.tensor(true_sparsity)
    results[Out_iter,8,:] = torch.tensor(effective_sparsity)

#             df=pd.read_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv')
#             new_row = {
#                         'tau': tau, 
#                         'gamma': gamma, 
#                         'rho': rho, 
#                         'alpha': alpha,
#                         'loss1': loss1[niter-1], 
#                         'loss2': loss2[niter-1], 
#                         'accuracy_train': accuracy_train[niter-1],
#                         'accuracy_test': accuracy_test[niter-1], 
#                         'max_accuracy_train': max(accuracy_train),
#                         'max_accuracy_test': max(accuracy_test),
#                         'time': time1[niter-1], 
#                         'BACC_train': bacc_train[niter-1],
#                         'BACC_test': bacc_test[niter-1],
#                         'max_BACC_train': max(bacc_train),
#                         'max_BACC_test': max(bacc_test),
#                         'Sparsity': sparsity,
#                         'seed' : seed
#                     }

#             df=df.append(new_row,ignore_index=True)
#             df.to_csv('/home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/differernt gamma rho/result.csv',index=False)

filename="Orthogonal_" + "niter_"+ str(niter) + "Sparsity_" + str(sparsity) +"tau_" + str(tau) + "gamma_" + str(gamma) + \
                "rho_" + str(rho) + "alpha_" + str(alpha)+ ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/Sparse/LeNet300_100/Flare(BCD method)/different init/
savemat (filename, {'results': torch.Tensor.numpy(results)})

0.9 200 100 1 1
Train on 8114 samples, validate on 44689 samples
Epoch 1 / 1000 
 - time: 0.029474496841430664 - sq_loss: 397717.6989335023 - tot_loss: 397717.90800162096 - acc: 0.8009613014542766 - val_acc: 0.8072903846584171 - bacc_train: 0.8009613014542765 - bacc_test: 0.860558391029671 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 2 / 1000 
 - time: 0.028175830841064453 - sq_loss: 390033.2415263759 - tot_loss: 390033.98336951394 - acc: 0.8009613014542766 - val_acc: 0.8064848172928462 - bacc_train: 0.8009613014542765 - bacc_test: 0.865839526909318 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 3 / 1000 
 - time: 0.028130769729614258 - sq_loss: 382543.9146260285 - tot_loss: 382545.42082533694 - acc: 0.8008380576780872 - val_acc: 0.817494237955649 - bacc_train: 0.8008380576780872 - bacc_test: 0.8665699530444333 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 4 / 1000 
 - time: 0

Epoch 31 / 1000 
 - time: 0.02983546257019043 - sq_loss: 222811.96556541303 - tot_loss: 222812.5386312971 - acc: 0.8175992112398324 - val_acc: 0.8372977690259348 - bacc_train: 0.8175992112398325 - bacc_test: 0.8756240665655622 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 32 / 1000 
 - time: 0.026848316192626953 - sq_loss: 218517.41748029375 - tot_loss: 218517.9962220624 - acc: 0.8179689425684002 - val_acc: 0.8376781758374544 - bacc_train: 0.8179689425684002 - bacc_test: 0.8765793064499889 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 33 / 1000 
 - time: 0.0268857479095459 - sq_loss: 214305.32614237408 - tot_loss: 214305.91061321268 - acc: 0.8180921863445896 - val_acc: 0.8379466982926447 - bacc_train: 0.8180921863445896 - bacc_test: 0.8767175202667634 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 34 / 1000 
 - time: 0.026849985122680664 - sq_loss: 210174.1379847413 - tot_loss:

Epoch 63 / 1000 
 - time: 0.026990652084350586 - sq_loss: 119475.02902210492 - tot_loss: 119476.4694086449 - acc: 0.8233916687207296 - val_acc: 0.8454653270379735 - bacc_train: 0.8233916687207297 - bacc_test: 0.8821063810911097 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 64 / 1000 
 - time: 0.026847362518310547 - sq_loss: 117169.7840027949 - tot_loss: 117171.21092257627 - acc: 0.8236381562731082 - val_acc: 0.8456219651368345 - bacc_train: 0.8236381562731082 - bacc_test: 0.8821870058175616 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 65 / 1000 
 - time: 0.02685260772705078 - sq_loss: 114908.99076620433 - tot_loss: 114910.40459357877 - acc: 0.8245008627064333 - val_acc: 0.8462708944035445 - bacc_train: 0.8245008627064333 - bacc_test: 0.8829007410300982 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 66 / 1000 
 - time: 0.02690911293029785 - sq_loss: 112691.79406232333 - tot_los

Epoch 95 / 1000 
 - time: 0.026818275451660156 - sq_loss: 64043.261421728115 - tot_loss: 64044.775844139294 - acc: 0.8273354695587872 - val_acc: 0.8534314932086196 - bacc_train: 0.8273354695587873 - bacc_test: 0.8865864428107528 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 96 / 1000 
 - time: 0.02685713768005371 - sq_loss: 62807.297176100496 - tot_loss: 62808.901890582965 - acc: 0.8277052008873552 - val_acc: 0.8537223925350758 - bacc_train: 0.8277052008873552 - bacc_test: 0.8867361744455919 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 97 / 1000 
 - time: 0.02677774429321289 - sq_loss: 61595.183020311786 - tot_loss: 61596.90657930393 - acc: 0.8279516884397338 - val_acc: 0.8538790306339368 - bacc_train: 0.8279516884397338 - bacc_test: 0.886437080683379 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 98 / 1000 
 - time: 0.026860952377319336 - sq_loss: 60406.45906249481 - tot_loss

Epoch 127 / 1000 
 - time: 0.026845216751098633 - sq_loss: 34327.69352522964 - tot_loss: 34330.40738216035 - acc: 0.8315257579492236 - val_acc: 0.859741770905592 - bacc_train: 0.8315257579492236 - bacc_test: 0.8898344675049548 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 128 / 1000 
 - time: 0.026801347732543945 - sq_loss: 33665.21591592495 - tot_loss: 33667.917112551164 - acc: 0.8315257579492236 - val_acc: 0.8599655396182506 - bacc_train: 0.8315257579492236 - bacc_test: 0.8903293641742649 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 129 / 1000 
 - time: 0.026855945587158203 - sq_loss: 33015.52526759509 - tot_loss: 33018.19057585327 - acc: 0.8317722455016021 - val_acc: 0.860055047103314 - bacc_train: 0.8317722455016021 - bacc_test: 0.8903754354465231 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8959241706161137
Epoch 130 / 1000 
 - time: 0.026912927627563477 - sq_loss: 32378.374762106734 - tot_lo

...

Epoch 863 / 1000 
 - time: 0.02748703956604004 - sq_loss: 0.06832654272222596 - tot_loss: 0.8009689588237296 - acc: 0.8535863938871087 - val_acc: 0.8775090066906845 - bacc_train: 0.8535863938871087 - bacc_test: 0.905814547844169 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 864 / 1000 
 - time: 0.02703547477722168 - sq_loss: 0.0676985279724186 - tot_loss: 0.798145823736542 - acc: 0.8535863938871087 - val_acc: 0.8775090066906845 - bacc_train: 0.8535863938871087 - bacc_test: 0.905814547844169 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 865 / 1000 
 - time: 0.027098417282104492 - sq_loss: 0.06708026482603294 - tot_loss: 0.7955721333453715 - acc: 0.8535863938871087 - val_acc: 0.8775313835619504 - bacc_train: 0.8535863938871087 - bacc_test: 0.9058260656622337 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 866 / 1000 
 - time: 0.027193069458007812 - sq_loss: 0.06647156499604942 - t

Epoch 895 / 1000 
 - time: 0.02686619758605957 - sq_loss: 0.05232364705596696 - tot_loss: 0.7337863161960845 - acc: 0.8539561252156767 - val_acc: 0.878068428472331 - bacc_train: 0.8539561252156767 - bacc_test: 0.9057227748071179 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 896 / 1000 
 - time: 0.026891469955444336 - sq_loss: 0.05193717962311518 - tot_loss: 0.7318750468998532 - acc: 0.8539561252156767 - val_acc: 0.878068428472331 - bacc_train: 0.8539561252156767 - bacc_test: 0.9057227748071179 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 897 / 1000 
 - time: 0.026996374130249023 - sq_loss: 0.051556339647290575 - tot_loss: 0.7300115913915783 - acc: 0.8539561252156767 - val_acc: 0.8781355590861286 - bacc_train: 0.8539561252156767 - bacc_test: 0.9057573282613116 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 898 / 1000 
 - time: 0.026875734329223633 - sq_loss: 0.05118106024128444

Epoch 927 / 1000 
 - time: 0.026890993118286133 - sq_loss: 0.042340744374382934 - tot_loss: 0.6841484664053481 - acc: 0.8539561252156767 - val_acc: 0.8784935890263823 - bacc_train: 0.8539561252156767 - bacc_test: 0.9059416133503444 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 928 / 1000 
 - time: 0.02683711051940918 - sq_loss: 0.04209540306861786 - tot_loss: 0.682771268947521 - acc: 0.8539561252156767 - val_acc: 0.8785607196401799 - bacc_train: 0.8539561252156767 - bacc_test: 0.905976166804538 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 929 / 1000 
 - time: 0.027025938034057617 - sq_loss: 0.04185339317210379 - tot_loss: 0.6813032773670802 - acc: 0.8539561252156767 - val_acc: 0.8785383427689141 - bacc_train: 0.8539561252156767 - bacc_test: 0.9059646489864734 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 930 / 1000 
 - time: 0.026871204376220703 - sq_loss: 0.04161464695492841

Epoch 959 / 1000 
 - time: 0.026848316192626953 - sq_loss: 0.03590306817479912 - tot_loss: 0.6435692205720964 - acc: 0.8543258565442445 - val_acc: 0.8790530108080288 - bacc_train: 0.8543258565442444 - bacc_test: 0.906229558801958 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 960 / 1000 
 - time: 0.02691364288330078 - sq_loss: 0.035741569985372444 - tot_loss: 0.6424012737699041 - acc: 0.8543258565442445 - val_acc: 0.8790753876792947 - bacc_train: 0.8543258565442444 - bacc_test: 0.9062410766200226 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 961 / 1000 
 - time: 0.026858806610107422 - sq_loss: 0.03558206415802435 - tot_loss: 0.6413259672370918 - acc: 0.8542026127680552 - val_acc: 0.8791425182930923 - bacc_train: 0.8542026127680552 - bacc_test: 0.9062756300742162 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 962 / 1000 
 - time: 0.02723383903503418 - sq_loss: 0.03542450802413671

Epoch 991 / 1000 
 - time: 0.026862382888793945 - sq_loss: 0.031587363782408284 - tot_loss: 0.6102353916876535 - acc: 0.8540793689918659 - val_acc: 0.8795900557184094 - bacc_train: 0.854079368991866 - bacc_test: 0.9057465494581778 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 992 / 1000 
 - time: 0.026906967163085938 - sq_loss: 0.031476666004825904 - tot_loss: 0.6092348442275307 - acc: 0.8540793689918659 - val_acc: 0.8796795632034728 - bacc_train: 0.854079368991866 - bacc_test: 0.9057926207304359 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 993 / 1000 
 - time: 0.0269315242767334 - sq_loss: 0.03136719962006792 - tot_loss: 0.6083092368966566 - acc: 0.8540793689918659 - val_acc: 0.8796795632034728 - bacc_train: 0.854079368991866 - bacc_test: 0.9057926207304359 - true_sparsity: 0.8957345971563981 - effective_sparsity: 0.8961374407582938
Epoch 994 / 1000 
 - time: 0.026932716369628906 - sq_loss: 0.031258940173284254

### Visualization of training results

In [ ]:
plt.figure()
plt.plot(np.arange(1,niter+1), loss2)
plt.yscale('log',base=2)
plt.title('training loss')

plt.figure()
plt.plot(np.arange(1,niter+1), accuracy_train)
plt.plot(np.arange(1,niter+1), accuracy_test)
plt.title('accuracy')